#### 기본적인 URLparse를 위한 구조

내가 원하는 seed의 dom 트리를 만들어준후 urljoin 을 해줌,

물론 if문으로 href를 attribute 하지않거나 길이가 3보다 짧은거 걸러줘

In [82]:
import requests
from bs4 import BeautifulSoup

In [118]:
def download(method ,url, param=None, data=None,timeout=1, maxetries=3): ##없을수도 있는 것에 None // header 는 빠진형태
    try : 
        resp = requests.request(method, url, params=param, data=data, headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500<= e.response.status_code <600 and maxetries>0:
            print(maxetries)
            time.sleep(timeout)
            print("재시도")
            download(method,url,param,data,timeout,maxetries-1)
        else :
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [119]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "html.parser")
    return [(requests.compat.urljoin(seed, _["href"])) 
            for _ in dom.find_all("tr td#page_list a") if _.has_attr('href') and len(_["href"])>3 ]

In [120]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
data = {"id" : "freeboard"}
html = download("get",url, data)
dom = BeautifulSoup(html.text, "html.parser")

#http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221

In [91]:
dom

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="PPOMPPU CO." name="author"/>
<meta content="뽐뿌" name="description"/>
<meta content="" name="keywords"/>
<script async="async" src="//www.googletagservices.com/tag/js/gpt.js"></script>
<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>
<!--
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />
-->
<meta content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" property="og:image">
<meta content="뽐뿌" property="og:site_name">
<title>뽐뿌 - 자유게시판</title><!--<link href='ht

In [100]:
dom.select("#page_list a")

[<a href="/zboard/zboard.php?id=freeboard&amp;page=2&amp;divpage=1221" onfocus="blur()">2</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=3&amp;divpage=1221" onfocus="blur()">3</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=4&amp;divpage=1221" onfocus="blur()">4</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=5&amp;divpage=1221" onfocus="blur()">5</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=6&amp;divpage=1221" onfocus="blur()">6</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=7&amp;divpage=1221" onfocus="blur()">7</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=8&amp;divpage=1221" onfocus="blur()">8</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=9&amp;divpage=1221" onfocus="blur()">9</a>,
 <a href="/zboard/zboard.php?id=freeboard&amp;page=10&amp;divpage=1221" onfocus="blur()">10</a>,
 <a class="page_next" href="/zboard/zboard.php?id=freeboard&amp;page=11&amp;divpage=1221" onfocus="blur()">다음<img src="/images/main/arrow_right

In [121]:
from bs4 import BeautifulSoup

def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "html.parser")
    return [requests.compat.urljoin(seed, _.find_parent()["href"]) 
            for _ in dom.select("font.list_title") 
            if _.find_parent() and _.find_parent().has_attr("href")]

In [122]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, param ={"id" : "freeboard"})
dom = BeautifulSoup(html.text, "html.parser")

seed = list()
seed.extend([requests.compat.urljoin(url, _["href"])
             for _ in dom.select("#page_list a")])
seen = list()
queue = list()

while seed:
    baseURL = seed.pop(0) 
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

Queue : 30, Seen : 1
Queue : 60, Seen : 2
Queue : 90, Seen : 3
Queue : 120, Seen : 4
Queue : 150, Seen : 5
Queue : 180, Seen : 6
Queue : 210, Seen : 7
Queue : 240, Seen : 8
Queue : 270, Seen : 9
Queue : 300, Seen : 10


In [133]:
def parseContent(url):
    html = download("get",url)
    dom = BeautifulSoup(html.text, "html.parser")
    
    return {"title":dom.select_one("font.view_title2").text.strip(),
           "body":_dom.select_one(".pic_bg .han").text.strip(),
           "comments":[_.text.strip()
                      for _ in dom.select("div[id^=comment]")]}

In [127]:
dom.select_one("td.pic_bg han")

AttributeError: 'NoneType' object has no attribute 'text'

In [110]:
url_ = "http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=11&divpage=1221&no=6543007"
html_ = download("get", url_)
dom_ = BeautifulSoup(html_.text, "html.parser")
dom_

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="PPOMPPU CO." name="author"/>
<meta content="뽐뿌" name="description"/>
<meta content="" name="keywords"/>
<script async="async" src="//www.googletagservices.com/tag/js/gpt.js"></script>
<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>
<!--
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />
-->
<meta content="뽐뿌" property="og:site_name">
<title>뽐뿌 - 자유게시판</title><!--<link href='http://fonts.googleapis.com/css?family=Noto+Sans' rel='stylesheet' type='text/css'>-->
<link 

In [116]:
_url = "http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=11&divpage=1221&no=6543007"
_html = download("get", _url)
_dom = BeautifulSoup(_html.text,"html.parser")

In [132]:
_dom.select_one(".pic_bg .han").text

'\n샐러드에 양상추 방토 무슨잡초하나랑 씨리얼 몇개랑 드레싱?그거해서 먹는데\n드레싱 고것이 살이찌는가요???\xa0'

In [97]:
import sqlite3
con = sqlite3.connect("plzgohome2.db")
cur = con.cursor()

In [98]:
parseContent("http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=11&divpage=1221&no=6543002")

AttributeError: 'NoneType' object has no attribute 'text'

In [80]:
dom.select(".pic_bg")

[]

In [134]:
cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    )
""")
con.commit()

In [135]:
while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3
        (title,content)
        VALUES(?,?)
    """,[content["title"],content["body"]])
    con.commit()
    
    cur.execute("""
        SELECT id
        FROM table3
        WHERE title=? AND content=?;
    """,[content["title"],content["body"]])
    
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4
            (table3_id , comment)
            VALUES(?,?)
        """,[table3_id,comment])
    con.commit()

KeyboardInterrupt: 

In [68]:
dom

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="PPOMPPU CO." name="author"/>
<meta content="뽐뿌" name="description"/>
<meta content="" name="keywords"/>
<script async="async" src="//www.googletagservices.com/tag/js/gpt.js"></script>
<script>var googletag = googletag || {};googletag.cmd = googletag.cmd || [];</script>
<!--
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, target-densitydpi=device-dpi" />
-->
<meta content="http://www.ppomppu.co.kr/images/icon_app_20160427.png" property="og:image">
<meta content="뽐뿌" property="og:site_name">
<title>뽐뿌 - 자유게시판</title><!--<link href='ht